In [1]:
#hopeのページからe-Learningの小テストの評点csvと目標設定のcsvをスクレイピングしてダウンロードするプログラム
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from dotenv import load_dotenv
from datetime import datetime
import shutil

load_dotenv()

# 環境変数
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")
SELENIUM_HOST = os.getenv("SELENIUM_HOST", "selenium")

# フォルダパス
SELENIUM_DOWNLOAD_DIR = "/home/seluser/Downloads" #Downloadsフォルダにseluserの書き込み権限がないので出力できない->した
JUPYTER_SHARED_DIR = "/home/jovyan/work/shared_downloads"
TODAY_STR = datetime.now().strftime("%Y%m%d")
TARGET_FOLDER = os.path.join(JUPYTER_SHARED_DIR, TODAY_STR)

# 小テストCSVのURL一覧
TEST_CSV_URLS = [
    "https://hope.fun.ac.jp/grade/export/txt/index.php?id=2059", #グループ1（keitest4: 全支援あり）
    "https://hope.fun.ac.jp/grade/export/txt/index.php?id=2409", #グループ2（keilabtest5: 支援なし）
    "https://hope.fun.ac.jp/grade/export/txt/index.php?id=2412", #グループ3（keilabtest7: 自律性のみ）
]

    #目標設定のcsvをダウンロードするリンク先の一覧配列
     # subject_purpose_url = {
     #     'Linux': {
     #         'keitest4': [
     #             ''
     #         ],
     #         'keilabtest5': [
     #             ''
     #         ]
     #     },
     #     'Network': {
     #         'keitest4': [
     #             ''
     #         ],
     #         'keilabtest5': [
     #             ''
     #         ]
     #     },
     #     'Cloud': {
     #         'keitest4': [
     #             ''
     #         ],
     #         'keilabtest5': [
     #             ''
     #         ]
     #     },
     #     'Java': {
     #         'keitest4': [
     #             ''
     #         ],
     #         'keilabtest5': [
     #             ''
     #         ]
     #     },
     #     'Algo': {
     #         'keitest4': [
     #             ''
     #         ],
     #         'keilabtest5': [
     #             ''
     #         ]
     #     },         
     #     'UML': {
     #         'keitest4': [
     #             ''
     #         ],
     #         'keilabtest5': [
     #             ''
     #         ]
     #     }, 
     #     'SystemDev': {
     #         'keitest4': [
     #             ''
     #         ],
     #         'keilabtest5': [
     #             ''
     #         ]
     #     },  
     #     'Management': {
     #         'keitest4': [
     #             ''
     #         ],
     #         'keilabtest5': [
     #             ''
     #         ]
     #     },           
     # }

def setup_driver():
    options = Options()
    options.add_experimental_option("prefs", {
        "download.default_directory": SELENIUM_DOWNLOAD_DIR,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--headless=new")
    options.add_argument("--remote-debugging-port=9222")

    return webdriver.Remote(
        command_executor=f"http://{SELENIUM_HOST}:4444/wd/hub",
        options=options
    )

def login_to_hope(driver):
    login_start_url = "https://hope.fun.ac.jp/local/hope/login.php"
    driver.get(login_start_url)

    print("⌛ 「未来大の学生・教職員」リンクをクリック")
    driver.find_element(By.LINK_TEXT, "未来大の学生・教職員 FUN Students & Staff").click()

    print("⌛ ユーザー名・パスワード入力待機中")
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.NAME, "username")))
    driver.find_element(By.NAME, "username").send_keys(USERNAME)
    driver.find_element(By.NAME, "password").send_keys(PASSWORD)
    driver.find_element(By.CSS_SELECTOR, "input[type='submit']").click()

    WebDriverWait(driver, 15).until(EC.url_changes(login_start_url))
    print("✅ ログイン成功")

def download_csv_from_urls(driver, urls):
    for i, url in enumerate(urls):
        print(f"\n➡️ ({i+1}) URLアクセス中: {url}")
        driver.get(url)

        try:
            download_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.ID, "id_submitbutton"))
            )
            time.sleep(2)
            download_button.click()
            print("⬇️ ダウンロード中...（4秒待機）")
            time.sleep(4)
        except Exception as e:
            print(f"❌ ダウンロード失敗: {e}")

def move_downloaded_files():
    if not os.path.exists(TARGET_FOLDER):
        os.makedirs(TARGET_FOLDER)

    print("📂 ファイル移動開始...")
    for filename in os.listdir(JUPYTER_SHARED_DIR):
        file_path = os.path.join(JUPYTER_SHARED_DIR, filename)
        if os.path.isfile(file_path):
            target_path = os.path.join(TARGET_FOLDER, filename)
            shutil.move(file_path, target_path)
            print(f"✅ {filename} を {TARGET_FOLDER} に移動")

def main():
    driver = setup_driver()
    try:
        login_to_hope(driver)
        download_csv_from_urls(driver, TEST_CSV_URLS)
        move_downloaded_files()
        print("🎉 全ダウンロード完了")
    finally:
        driver.quit()

if __name__ == "__main__":
    main()


⌛ 「未来大の学生・教職員」リンクをクリック
⌛ ユーザー名・パスワード入力待機中
✅ ログイン成功

➡️ (1) URLアクセス中: https://hope.fun.ac.jp/grade/export/txt/index.php?id=2059
⬇️ ダウンロード中...（4秒待機）

➡️ (2) URLアクセス中: https://hope.fun.ac.jp/grade/export/txt/index.php?id=2409
⬇️ ダウンロード中...（4秒待機）

➡️ (3) URLアクセス中: https://hope.fun.ac.jp/grade/export/txt/index.php?id=2412
⬇️ ダウンロード中...（4秒待機）
📂 ファイル移動開始...
✅ keitest4 評定-20250512_1209-comma_separated.csv を /home/jovyan/work/shared_downloads/20250512 に移動
✅ keilabtest7 評定-20250512_1209-comma_separated.csv を /home/jovyan/work/shared_downloads/20250512 に移動
✅ keilabtest5 評定-20250512_1209-comma_separated.csv を /home/jovyan/work/shared_downloads/20250512 に移動
🎉 全ダウンロード完了
